**1. Importing the dependencies**

## 이탈률 높은 고객 10% 고르기? ==> 이건 주로 어떻게 하는지 보기 

In [430]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

from sklearn.model_selection import GridSearchCV


import pandas as pd
import numpy as np
import pickle

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # SMOTE용 Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score


**2. Data Loading and Understanding**

In [431]:
# load teh csv data to a pandas dataframe
df = pd.read_csv("customer_info.csv")

In [432]:
df.shape

(7043, 21)

In [433]:
""" 
고객 특성 → gender, SeniorCitizen, Partner(고객이 배우자(파트너)가 있는지 여부), Dependents(부양가족(아이, 부모 등)이 있는지 여부)
서비스 사용 여부 → PhoneService(고객이 전화 서비스를 이용하는지 여부), MultipleLines, InternetService, OnlineSecurity, DeviceProtection, TechSupport, StreamingTV, StreamingMovies
계약/결제 관련 → Contract(계약 형태), PaperlessBilling(종이 청구서 대신 전자 청구서를 사용하는지 여부), PaymentMethod
금액 관련 → MonthlyCharges, TotalCharges, tenure(고객이 해당 회사와 계약을 유지한 개월 수)
타겟 변수 → Churn : Yes = 서비스를 해지한 고객, No = 유지 중인 고객
"""

' \n고객 특성 → gender, SeniorCitizen, Partner(고객이 배우자(파트너)가 있는지 여부), Dependents(부양가족(아이, 부모 등)이 있는지 여부)\n서비스 사용 여부 → PhoneService(고객이 전화 서비스를 이용하는지 여부), MultipleLines, InternetService, OnlineSecurity, DeviceProtection, TechSupport, StreamingTV, StreamingMovies\n계약/결제 관련 → Contract(계약 형태), PaperlessBilling(종이 청구서 대신 전자 청구서를 사용하는지 여부), PaymentMethod\n금액 관련 → MonthlyCharges, TotalCharges, tenure(고객이 해당 회사와 계약을 유지한 개월 수)\n타겟 변수 → Churn : Yes = 서비스를 해지한 고객, No = 유지 중인 고객\n'

In [434]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [435]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [436]:
# churn_rate = df['Churn'].value_counts(normalize=True) * 100
# print(churn_rate)

In [437]:
pd.set_option("display.max_columns", None) 
#Pandas 데이터프레임을 출력할 때 한 화면에 모든 컬럼을 다 보이게 해주는 옵션

In [438]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [439]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [440]:
# dropping customerID column as this is not required for modelling
# df = df.drop(columns=["customerID"])

In [441]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [442]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [443]:
print(df["gender"].unique())
#데이터프레임 df에서 "gender" 컬럼만 선택.해당 컬럼에 들어있는 고유값(unique values) 을 중복 없이 반환

['Female' 'Male']


In [444]:
print(df["SeniorCitizen"].unique())

[0 1]


In [445]:
print(df.head())
print(df.dtypes)

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             DSL            Yes           No   
4                No     Fiber optic             No           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No          No        

In [446]:
# printing the unique values in all the columns
# 숫자형 변수 빼고 나머지 컬럼들의 고유한 값들(중복빼고)을 출력


numerical_features_list = ["tenure", "MonthlyCharges", "TotalCharges"]


for col in df.columns:
  if col not in numerical_features_list:
    print(col, df[col].unique())
    print("-"*50)



customerID ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
--------------------------------------------------
gender ['Female' 'Male']
--------------------------------------------------
SeniorCitizen [0 1]
--------------------------------------------------
Partner ['Yes' 'No']
--------------------------------------------------
Dependents ['No' 'Yes']
--------------------------------------------------
PhoneService ['No' 'Yes']
--------------------------------------------------
MultipleLines ['No phone service' 'No' 'Yes']
--------------------------------------------------
InternetService ['DSL' 'Fiber optic' 'No']
--------------------------------------------------
OnlineSecurity ['No' 'Yes' 'No internet service']
--------------------------------------------------
OnlineBackup ['Yes' 'No' 'No internet service']
--------------------------------------------------
DeviceProtection ['No' 'Yes' 'No internet service']
---------------------------------------

In [447]:
print(df.isnull().sum())

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [448]:
df["TotalCharges"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7043 entries, 0 to 7042
Series name: TotalCharges
Non-Null Count  Dtype 
--------------  ----- 
7043 non-null   object
dtypes: object(1)
memory usage: 55.2+ KB


In [449]:
# df["TotalCharges"] = df["TotalCharges"].astype(float)
# TotalCharges 컬럼이 원래 문자열이라 숫자로 바꾸려고 시도했음 → 실패 (빈칸 때문).
# 빈칸 " " 값들을 찾아내서 "0.0"으로 치환.
# 전체 컬럼을 float으로 변환.

In [450]:
#삭제하기 df[df["TotalCharges"]==" "]# 

In [451]:
#삭제하기len(df[df["TotalCharges"]==" "])

In [452]:
#삭제하기df["TotalCharges"] = df["TotalCharges"].replace({" ": "0.0"})

In [453]:
#삭제하기df["TotalCharges"] = df["TotalCharges"].astype(float)

In [454]:
# checking the class distribution of target column
print(df["Churn"].value_counts())

Churn
No     5174
Yes    1869
Name: count, dtype: int64


**Insights:**
1. Customer ID removed as it is not required for modelling
2. No mmissing values in the dataset
3. Missing values in the TotalCharges column were replaced with 0
4. Class imbalance identified in the target

**3. Exploratory Data Analysis (EDA)**

In [455]:
df.shape

(7043, 21)

In [456]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [457]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [458]:
df.describe()
# 금액과 같은 수치가 있는것만 통계적 요약정보를 줌. 

# 1. SeniorCitizen (시니어 여부, 0=아님, 1=예)
# count: 7043명 고객 데이터
# mean: 0.162 → 약 16.2%가 시니어(1), 나머지 83.8%는 비시니어(0)
# std: 0.368 → 이진변수이므로 분산 작음
# min~max: 0~1 (정상)
# 👉 고객의 대부분은 비시니어.

# 2. tenure (가입 개월 수)
# 범위: 0 ~ 72개월
# 중앙값: 29개월 → 절반 고객은 29개월 이하, 절반은 그 이상
# 평균: 32개월 → 다양한 분포, 신규 고객도 있고 장기 고객도 있음

# 3. MonthlyCharges (월 요금)
# 범위: $18 ~ $119
# 중앙값: $70 → 대부분 고객은 70달러 근처
# 평균: $65 → 저가 요금제 고객도 많고 고가 요금제 고객도 일부 있음

# 4. TotalCharges (총 요금)  --> 계약 월 수  x 월별 비용 
# 범위: $0 ~ $8685
# 중앙값: $1395
# 평균: $2280
# 분산이 매우 큼 (고객이 오래 다녔는지 여부에 따라 크게 차이남)

# count 전체중에 %인지 보면됨. 


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [459]:
# 🔎 tenure 해석
# mean = 32.37개월
# → 평균적으로 고객들은 약 32개월(2년 8개월) 동안 서비스를 유지.
# std = 24.56개월
# → 표준편차가 25개월 정도라서, 대략적으로
# 평균 ± 1표준편차 = (약 8개월 ~ 57개월)
# 범위 안에 고객들의 약 68% 가 분포한다고 볼 수 있음.
# 50% (median) = 29개월
# → 가입자의 절반은 29개월 이하, 절반은 29개월 이상 유지.
# 75% = 55개월
# → 전체 고객 중 75%는 55개월 이하로 다녔고, 나머지 25%는 그 이상 다녔음.
# max = 72개월
# → 최장 고객은 6년 동안 유지.

# 📌 예시 (tenure 기준)
# 25% = 9개월
# → 고객 중 25%는 가입 9개월 이하에서 이탈하거나 아직 유지 중
# 50% = 29개월
# → 고객의 절반은 29개월 이하, 절반은 29개월 이상
# 75% = 55개월
# → 고객의 75%는 55개월 이하, 상위 25% 충성 고객은 55개월 이상 유지 

# ✅ 정리
# 읽을 때는 말씀하신 것처럼:
# 25% 값 = “하위 25% 그룹의 상한선”
# 50% 값 = “전체를 절반으로 나누는 기준점”
# 75% 값 = “상위 25% 그룹의 하한선”
# 👉 그래서 “25% 미만의 사람들은…” / “50%는 절반의 고객들은…” / “75%는 상위 25% 고객은…” 이런 식으로 해석하면 맞습니다 ✔️

<!-- ```📌 예시 (tenure 기준)
25% = 9개월
→ 고객 중 25%는 가입 9개월 이하에서 이탈하거나 아직 유지 중
50% = 29개월
→ 고객의 절반은 29개월 이하, 절반은 29개월 이상
75% = 55개월
→ 고객의 75%는 55개월 이하, 상위 25% 충성 고객은 55개월 이상 유지 


✅ 정리
읽을 때는 말씀하신 것처럼:
25% 값 = “하위 25% 그룹의 상한선”
50% 값 = “전체를 절반으로 나누는 기준점”
75% 값 = “상위 25% 그룹의 하한선”
👉 그래서 “25% 미만의 사람들은…” / “50%는 절반의 고객들은…” / “75%는 상위 25% 고객은…” 이런 식으로 해석하면 맞습니다 ✔️ -->



**Numerical Features - Analysis**

Understand the distribution of teh numerical 

데이터셋 안의 숫자형 변수들이 어떻게 분포되어 있는지 이해

In [460]:
# def plot_histogram(df, column_name):

#   plt.figure(figsize=(5, 3))
#   sns.histplot(df[column_name], kde=True)
#   plt.title(f"Distribution of {column_name}")

#   # 선택한 컬럼의 평균과 중앙값 계산
#   col_mean = df[column_name].mean()
#   col_median = df[column_name].median()

#   # add vertical lines for mean and median
#   plt.axvline(col_mean, color="red", linestyle="--", label="Mean")
#   plt.axvline(col_median, color="green", linestyle="-", label="Median")

#   plt.legend()

#   plt.show()

In [461]:
# plot_histogram(df, "tenure") # 가입 개월수 분포  

In [462]:
# plot_histogram(df, "MonthlyCharges")

In [463]:
# plot_histogram(df, "TotalCharges")

**Box plot for numerical features**

In [464]:
# def plot_boxplot(df, column_name):

#   plt.figure(figsize=(5, 3))
#   sns.boxplot(y=df[column_name])
#   plt.title(f"Box Plot of {column_name}")
#   plt.ylabel(column_name)
#   plt.show

In [465]:
# plot_boxplot(df, "tenure")

In [466]:
# plot_boxplot(df, "MonthlyCharges")

In [467]:
# plot_boxplot(df, "TotalCharges")

**Correlation Heatmap for numerical columns**

In [468]:
# # correlation matrix - heatmap
# plt.figure(figsize=(8, 4))
# sns.heatmap(df[["tenure", "MonthlyCharges", "TotalCharges"]].corr(), annot=True, cmap="coolwarm", fmt=".2f")
# plt.title("Correlation Heatmap")
# plt.show()

<!-- 📊 해석
tenure ↔ TotalCharges : 0.83 (강한 양의 상관관계)
가입 기간(tenure)이 길수록 총 요금(TotalCharges)도 높다.
당연히 tenure × MonthlyCharges ≈ TotalCharges 구조이기 때문에 높은 상관성이 있음.

MonthlyCharges ↔ TotalCharges : 0.65 (중간 정도 양의 상관관계)
월 요금이 높으면 총 요금도 대체로 높다.
하지만 tenure가 짧으면 총 요금이 낮을 수도 있으므로 완벽한 상관관계는 아님.

tenure ↔ MonthlyCharges : 0.25 (약한 양의 상관관계)
가입 기간과 월 요금은 거의 관련이 없음.
오래된 고객이라고 해서 반드시 요금을 많이 내는 건 아니다. (요금제와 무관) -->

Categorical features - Analysis

In [469]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [470]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Countplot for categorical columns

In [471]:
# object_cols = df.select_dtypes(include="object").columns.to_list()

# object_cols = ["SeniorCitizen"] + object_cols

# for col in object_cols:
#   plt.figure(figsize=(5, 3))
#   sns.countplot(x=df[col])
#   plt.title(f"Count Plot of {col}")
#   plt.show()

In [472]:
from collections import Counter
from imblearn.over_sampling import SMOTE


# ---------------------------
# 1. Custom Transformer: Risk Feature Generator /즉, “이 변환기가 기억해야 하는 기준값을 담을 주머니”를 준비하는 단계예요.
# 나중에 transform() 단계에서 새로운 데이터(Test set, 실시간 고객 데이터 등)를 변환할 때,
# 이 기준값을 그대로 써야 데이터 누설(leakage)을 막을 수 있음
# ---------------------------
class RiskFeatureGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, stats=None): #__init__: 통계값 저장 공간 준비 #파이프라인 안에서 객체를 만들 때 실행되는 “생성자(constructor)”/ 처음에는 None이지만, fit() 단계에서 채워집니다.
        self.stats = stats # stats 라는 속성을 클래스 내부에 저장합니다.stats는 나중에 기준값(중위수, 사분위수) 을 담을 공간이에요.
    
    def fit(self, X, y=None): #fit 단계 = 훈련 데이터에서 기준값을 학습하는 단계
        # train에서 기준값 저장
        # 여기서 하는 일:
        # MonthlyCharges의 중위수(median) 계산(median_monthly: 고객을 “월 요금 기준 High/Low Value”로 나눌 기준)
        # TotalCharges의 중위수(median) 계산(median_monthly: 고객을 “월 요금 기준 High/Low Value”로 나눌 기준)
        # tenure(가입 기간)의 25% 분위(Q1) 계산(짧을수록 이탈높음)
        # 이렇게 계산된 값을 self.stats 딕셔너리에 저장해 둡니다.
        # 즉, 훈련셋에서 “이탈 위험을 가르는 기준선”을 뽑아 두는 작업이에요.
        # 예: median_monthly = 65 / median_total = 1400 / q25_tenure = 12         
        self.stats = {
            "median_monthly": X["MonthlyCharges"].median(),
            "median_total": X["TotalCharges"].median(),
            "q25_tenure": X["tenure"].quantile(0.25)
        }
        return self
    

# 📌 기본 개념 아래 
# transform() 은 입력된 데이터셋(X)을 받아서 새로운 컬럼(파생변수) 을 추가하거나, 기존 값을 변환하는 역할을 합니다.
# 그런데 파이프라인 안에서 들어오는 X는 원본 참조(Reference) 일 수 있습니다.
# → 그대로 수정하면 원본 데이터 자체가 바뀌어버릴 수 있어요.
    def transform(self, X):
        X = X.copy() # 원본 데이터 X를 복사해서 사용 (원본 손상 방지).
        X['HighValue'] = ((X['MonthlyCharges'] > self.stats["median_monthly"]) | (X['TotalCharges'] > self.stats["median_total"])).astype(int) #월 요금이 중위수보다 크거나 총 요금이 중위수보다 크면 HighValue=1 (고가치 고객) 아니면 0
        X['Risk_Contract']  = (X['Contract'] == 'Month-to-Month').astype(int) # 계약 유형이 단기(월별) → 위험 고객 → 1
        X['Risk_Tenure']    = (X['tenure'] < self.stats["q25_tenure"]).astype(int) # 가입기간이 하위 25% (예: 12개월 미만) → 위험 고객 → 1
        X['Risk_Paperless'] = (X['PaperlessBilling'] == 'Yes').astype(int) # 종이 고지서 없이 전자 청구만 하는 고객 → 상대적으로 이탈이 쉬움 → 위험 고객 → 1 
        X['Risk_Fiber']     = (X['InternetService'] == 'Fiber optic').astype(int) # Fiber optic (경쟁사로 옮기기 쉽고 churn율 높은 서비스) → 위험 고객 → 1
        X['Risk_Echeck']    = (X['PaymentMethod'] == 'Electronic check').astype(int) # 전자결제(체크카드) 방식 → churn율 높은 결제수단 → 위험 고객 → 1
        
        risk_cols = ['Risk_Contract','Risk_Tenure','Risk_Paperless','Risk_Fiber','Risk_Echeck']
        X['RiskScore'] = X[risk_cols].sum(axis=1)
        X['HighRisk']  = (X['RiskScore'] >= 2).astype(int) 
        # RiskScore ≥ 2 (위험 요인 2개 이상) → HighRisk=1
        # RiskScore < 2 → HighRisk=0
        return X #파생변수까지 포함된 DataFrame 반환
    #위 5개 리스크 변수 합산해서 총 Risk 점수 계산 (0~5 사이 값).
    #예: Risk_Contract=1, Risk_Tenure=1, Risk_Paperless=0, Risk_Fiber=1, Risk_Echeck=0 → RiskScore=3



# ---------------------------
# 2. 데이터 준비
# ---------------------------
# 삭제하기 df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
# 삭제하기  df = df.dropna()

# " " 공백을 0.0 으로 치환 후 float 변환
df["TotalCharges"] = df["TotalCharges"].replace(" ", "0.0")
df["TotalCharges"] = df["TotalCharges"].astype(float)


X = df.drop(columns=["Churn", "customerID"])
y = df["Churn"].map({"No":0, "Yes":1})

# ---------------------------
# 3. 범주형/수치형 정의
# ---------------------------
categorical_cols = X.select_dtypes(include="object").columns.tolist()
numeric_cols = X.select_dtypes(include=["int64","float64"]).columns.tolist()

# ---------------------------
# 4. 전처리 파이프라인
# ---------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols) #passthrough : 아무 변화 없이 그대로 통과시킴
    ]
)

# ---------------------------
# 5. 모델 정의 (Voting 앙상블)
# ---------------------------
rfc = RandomForestClassifier(
    random_state=42,
    class_weight="balanced",
    n_estimators=200,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=2
)

xgb = XGBClassifier(
    random_state=42,
    scale_pos_weight=3,
    eval_metric="logloss"
)

lr = LogisticRegression(max_iter=1000, solver="liblinear", class_weight="balanced")


voting_clf = VotingClassifier(
    estimators=[("rf", rfc), ("xgb", xgb), ("lr", lr)],
    voting="soft"
)

# ---------------------------
# 6. 전체 파이프라인 (SMOTE 포함)
# ---------------------------
pipeline = ImbPipeline(steps=[
    ("risk", RiskFeatureGenerator()),  # 파생변수 생성
    ("preprocess", preprocessor),      # 인코딩
    ("smote", SMOTE(random_state=42)), # 이 단계가 바로 Churn 데이터(= 소수 클래스, y=1)를 오버샘플링 하는 부분/ 불균형 처리/오버샘플링 (학습 데이터에서만 적용) SMOTE는 학습용 데이터(fit)에만 적용돼요.
    ("model", voting_clf)              # 모델 /최종 학습 
])



# ---------------------------
# 7. Train/Test Split & 학습
# ---------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

pipeline.fit(X_train, y_train)

print("✅ 모델 학습 완료")




# SMOTE 불균형 전/후 확인
# from collections import Counter

# # Risk + Preprocess까지 거쳐서 수치형으로 변환
# X_train_processed = pipeline.named_steps["preprocess"].fit_transform(
#     pipeline.named_steps["risk"].fit_transform(X_train)
# )

# smote = pipeline.named_steps["smote"]

# print("SMOTE 전 y_train 분포:", Counter(y_train))
# X_res, y_res = smote.fit_resample(X_train_processed, y_train)
# print("SMOTE 후 y_train 분포:", Counter(y_res))

# # ---------------------------
# # 파이프라인 학습
# # ---------------------------
# pipeline.fit(X_train, y_train)
# print("✅ 모델 학습 완료")





# 1. 테스트셋 예측
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)[:, 1]

# 2. 평가 지표 출력
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))



# 8. 저장
# ---------------------------
with open("customer_churn_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)
print("✅ 파이프라인 저장 완료")


✅ 모델 학습 완료
Accuracy: 0.7679205110007097
Confusion Matrix:
 [[821 214]
 [113 261]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.79      0.83      1035
           1       0.55      0.70      0.61       374

    accuracy                           0.77      1409
   macro avg       0.71      0.75      0.72      1409
weighted avg       0.79      0.77      0.78      1409

ROC-AUC: 0.8396548606267276
✅ 파이프라인 저장 완료


In [473]:
# ✅ 정리
# RiskScore 같은 파생변수는 train/test 모두 똑같이 생성됨
# SMOTE는 train에서만 적용
# 전처리 이후 모델에 들어갈 때는 numpy array 로 변환돼 컬럼명이 없어지므로, 확인하려면 risk 단계나 preprocess.get_feature_names_out() 을 써야 함
# ✅ 보이지도 않는 Risk Score는 왜 파생변수로 만들었나?
# RiskScore는 단순히 보려고 만든 게 아니라, 도메인 지식을 반영해서 모델 성능과 해석력을 동시에 높이려는 것.
# Train/Test 둘 다 이 feature를 사용해서 학습/예측에 활용합니다.
# 모델이 스스로 찾아내기 어려운 조합을 사람이 설계해서 준 거라, 실제로 성능이 오를 수도 있습니다.

In [474]:
# 1. Risk 단계 이후 피처 이름 확보
X_train_risk = pipeline.named_steps["risk"].transform(X_train)
pipeline.named_steps["preprocess"].fit(X_train_risk)
feature_names = pipeline.named_steps["preprocess"].get_feature_names_out()

# 2. 개별 모델 꺼내기
rf_model  = pipeline.named_steps["model"].estimators_[0]  # RandomForest
xgb_model = pipeline.named_steps["model"].estimators_[1]  # XGB
lr_model  = pipeline.named_steps["model"].estimators_[2]  # LogisticRegression

# 3. 중요도 뽑기
import pandas as pd

rf_importance = pd.Series(rf_model.feature_importances_, index=feature_names)
xgb_importance = pd.Series(xgb_model.feature_importances_, index=feature_names)
lr_importance  = pd.Series(abs(lr_model.coef_[0]), index=feature_names)  # 절댓값으로 중요도 해석

print("랜덤포레스트 중요도 TOP 10:\n", rf_importance.sort_values(ascending=False).head(10))
print("XGB 중요도 TOP 10:\n", xgb_importance.sort_values(ascending=False).head(10))
print("로지스틱 중요도 TOP 10:\n", lr_importance.sort_values(ascending=False).head(10))


# ✅ 결론:
# 세 모델 모두 공통적으로 “단기 계약 + 특정 결제방식 + 보안/기술지원 여부”가 이탈의 핵심 요인임을 보여줌.
# RiskScore는 TOP에 안 떴지만, 설명가능성(Explainability)을 강화해주는 feature로 충분히 가치 있음

랜덤포레스트 중요도 TOP 10:
 cat__Contract_Month-to-month           0.096783
num__TotalCharges                      0.077598
cat__PaymentMethod_Electronic check    0.076403
num__tenure                            0.073950
cat__OnlineSecurity_No                 0.070129
cat__TechSupport_No                    0.060539
num__MonthlyCharges                    0.054365
cat__OnlineBackup_No                   0.047533
cat__Contract_Two year                 0.035573
cat__OnlineSecurity_Yes                0.026975
dtype: float64
XGB 중요도 TOP 10:
 cat__Contract_Month-to-month           0.335989
cat__InternetService_Fiber optic       0.093490
cat__OnlineSecurity_No                 0.057765
cat__Partner_Yes                       0.039982
cat__PaymentMethod_Electronic check    0.036914
cat__TechSupport_No                    0.035620
cat__Contract_Two year                 0.029951
cat__Dependents_Yes                    0.023153
cat__OnlineSecurity_Yes                0.021894
cat__InternetService_DSL            

**7. Load the saved  model and  build a Predictive System**

In [475]:
model_data = {
    "model": pipeline,               
    "features_names": X_train.columns.tolist()  
}
# model_data 라는 딕셔너리(dict) 를 만듭니다.
# "model" 키: 방금 학습된 파이프라인(pipeline) 자체를 저장 → 안에 RiskFeatureGenerator, OneHotEncoder, SMOTE, VotingClassifier 전부 포함됨
# "features_names" 키: 학습 당시 사용한 입력 변수 이름(X_train.columns.tolist())을 저장 → 나중에 예측할 때 컬럼 순서/이름이 맞는지 확인용
# 위는 묶는 작업 

import pickle
with open("model_data.pkl", "wb") as f:
    pickle.dump(model_data, f)
# pickle : Python 객체(모델, 딕셔너리, 리스트 등)를 파일로 저장하고 다시 불러올 수 있게 해주는 라이브러리
# "model_data.pkl" 이라는 파일을 쓰기 모드("wb") 로 열고 → model_data 딕셔너리를 저장
# → pickle 파일(model_data.pkl)로 저장
# 즉 두 코드는 “학습된 모델 + 메타데이터(피처 이름)” 를 패키징해서 저장하는 단계입니다.

In [476]:
print(model_data)

{'model': Pipeline(steps=[('risk',
                 RiskFeatureGenerator(stats={'median_monthly': np.float64(70.5),
                                             'median_total': np.float64(1394.925),
                                             'q25_tenure': np.float64(9.0)})),
                ('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'Onli...
                                                             max_bin=None,
                                                 

In [477]:
import pickle

with open("model_data.pkl", "rb") as f:
    model_data = pickle.load(f)

loaded_model = model_data["model"]
feature_names = model_data["features_names"]

print("✅ 모델과 feature_names 로드 완료")
print("Feature 개수:", len(feature_names))

# pickle.load(f) : 저장했던 객체를 다시 메모리로 불러옴
# "rb" : read binary → 바이너리 읽기 모드
# 결과적으로 model_data.pkl 안에 저장돼 있던 딕셔너리({"model": ..., "features_names": ...}) 가 복원됨
# loaded_model : 학습된 파이프라인 모델 객체 (RiskFeatureGenerator → Preprocess → SMOTE → VotingClassifier 구조 그대로 들어 있음)
# feature_names : 학습 당시 입력에 쓰였던 피처 이름 리스트


✅ 모델과 feature_names 로드 완료
Feature 개수: 19


In [478]:
feature_names

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges']

In [479]:
# 하드코딩 _ 신규 고객 예시 데이터
input_data = {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 1,
    "PhoneService": "No",
    "MultipleLines": "No phone service",
    "InternetService": "DSL",
    "OnlineSecurity": "No",
    "OnlineBackup": "Yes",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 29.85,
    "TotalCharges": 29.85
}

# DataFrame 변환
input_df = pd.DataFrame([input_data])

# 로드한 모델 사용
prediction = loaded_model.predict(input_df)
prediction_prob = loaded_model.predict_proba(input_df)

print("예측 결과:", "Churn (이탈)" if prediction[0] == 1 else "No Churn (유지)")
print("예측 확률:", prediction_prob) # predict_proba() : 확률 값 ([No Churn 확률, Churn 확률]) 예) [0.2, 0.8] → 이 고객은 80% 확률로 이탈할 것이라고 예측


예측 결과: Churn (이탈)
예측 확률: [[0.44869323 0.55130677]]


In [480]:
# 1. 새로운 고객 예시 (dict → DataFrame)
new_customer = pd.DataFrame([{
    "gender": "Male",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 2,
    "PhoneService": "Yes",
    "MultipleLines": "No",
    "InternetService": "Fiber optic",
    "OnlineSecurity": "No",
    "OnlineBackup": "No",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "Yes",
    "StreamingMovies": "Yes",
    "Contract": "Month-to-Month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 85.0,
    "TotalCharges": 170.0
}])

# 2. 이탈 확률 예측
y_prob = pipeline.predict_proba(new_customer)
print("예측 확률:", y_prob)   # 예: [[0.4486 0.5513]]
churn_pred = pipeline.predict(new_customer)[0]
print("예측 결과:", "Churn" if churn_pred==1 else "No Churn")

# 3. Risk 단계까지만 변환해서 파생변수 확인
X_risk = pipeline.named_steps["risk"].transform(new_customer)
# pipeline = ImbPipeline(steps=[에서 사용됨. 
#     ("risk", RiskFeatureGenerator()),   # ✅ 여기에 risk라는 이름으로 추가
#     ("preprocess", preprocessor),
#     ("smote", SMOTE(random_state=42)),
#     ("model", voting_clf)
# ])

# 4. Value 그룹 정의 (Median 기준 or 특정 Threshold)
median_monthly = X["MonthlyCharges"].median()
median_total   = X["TotalCharges"].median()

X_risk["ValueGroup"] = np.where(
    (X_risk["MonthlyCharges"] > median_monthly) | (X_risk["TotalCharges"] > median_total),
    "High Value", "Low Value"
)

# 5. Risk 그룹 정의 (RiskScore 활용)
X_risk["RiskGroup"] = np.where(X_risk["HighRisk"] == 1, "High Risk", "Low Risk")

# 6. 최종 세그먼트 조합
X_risk["Segment"] = X_risk["ValueGroup"] + " + " + X_risk["RiskGroup"]

print("최종 세그먼트:", X_risk["Segment"].iloc[0])

예측 확률: [[0.3300121 0.6699879]]
예측 결과: Churn
최종 세그먼트: High Value + High Risk


< Rule 기반 리스크 스코어 + 밸류 기준 = Rule-based Scoring> 

1.Value Score (spent 기준 = sales)

목적: 고객이 회사에 가져다주는 매출(가치)을 기준으로 High/Low Value 그룹 구분 : 합계 0~2

고객 쓴돈이 MonthlyCharges > 전체 고객의 median → +1

고객 쓴돈이 TotalCharges > 전체 고객의 median → +1

2.Risk Score (이탈 위험 점수) 이 고객이 언제든 이탈할 수 있는 “불안정 신호”를 계량화/즉, 서비스 계약이 불안정하거나 사용 행태가 이탈 위험 신호를 보이면 High Risk

-계산 방법 (각 조건 만족 시 +1) : 합계 = 0~5

Contract = Month-to-Month (단기 계약)

tenure < Q25 (가입 기간이 짧음)

PaperlessBilling = Yes (무통장/온라인 납부 → 충동적 이탈 가능성 ↑)

InternetService = Fiber optic (경쟁사 이동 쉬움)

PaymentMethod = Electronic check (이탈률 높은 결제수단)


| Value Score | Risk Score | Segment                    | 전략                               |
| ----------- | ---------- | -------------------------- | -------------------------------- |
| High (≥1)   | High (≥2)  | **High Value + High Risk** | VIP Retention (할인/번들 제공, 1순위 관리) |
| High (≥1)   | Low (<2)   | **High Value + Low Risk**  | Upsell (프리미엄 상품, 로열티 강화)         |
| Low (<1)    | High (≥2)  | **Low Value + High Risk**  | 저비용 캠페인 (필요시 churn 허용)           |
| Low (<1)    | Low (<2)   | **Low Value + Low Risk**   | Mass Campaign (자동화, 대규모 마케팅)     |


In [481]:
new_customer = pd.DataFrame([{
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "Yes",
    "tenure": 36,
    "PhoneService": "Yes",
    "MultipleLines": "Yes",
    "InternetService": "DSL",
    "OnlineSecurity": "Yes",
    "OnlineBackup": "Yes",
    "DeviceProtection": "Yes",
    "TechSupport": "Yes",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Two year",
    "PaperlessBilling": "No",
    "PaymentMethod": "Credit card (automatic)",
    "MonthlyCharges": 45.0,
    "TotalCharges": 1600.0
}])



# 2. 이탈 확률 예측
y_prob = pipeline.predict_proba(new_customer)
print("예측 확률:", y_prob)   # 예: [[0.4486 0.5513]]
churn_pred = pipeline.predict(new_customer)[0]
print("예측 결과:", "Churn" if churn_pred==1 else "No Churn")

# 3. Risk 단계까지만 변환해서 파생변수 확인
X_risk = pipeline.named_steps["risk"].transform(new_customer)

# 4. Value 그룹 정의 (Median 기준 or 특정 Threshold)
median_monthly = X["MonthlyCharges"].median()
median_total   = X["TotalCharges"].median()

X_risk["ValueGroup"] = np.where(
    (X_risk["MonthlyCharges"] > median_monthly) | (X_risk["TotalCharges"] > median_total),
    "High Value", "Low Value"
)

# 5. Risk 그룹 정의 (RiskScore 활용)
X_risk["RiskGroup"] = np.where(X_risk["HighRisk"] == 1, "High Risk", "Low Risk")

# 6. 최종 세그먼트 조합
X_risk["Segment"] = X_risk["ValueGroup"] + " + " + X_risk["RiskGroup"]

print("최종 세그먼트:", X_risk["Segment"].iloc[0])

예측 확률: [[0.97346111 0.02653889]]
예측 결과: No Churn
최종 세그먼트: High Value + Low Risk


In [482]:
new_customer = pd.DataFrame([{
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "Yes",
    "tenure": 24,                       # 충분히 오래 사용 → 안정적
    "PhoneService": "Yes",
    "MultipleLines": "Yes",
    "InternetService": "DSL",           # Fiber 아님 → 안정적
    "OnlineSecurity": "Yes",
    "OnlineBackup": "Yes",
    "DeviceProtection": "Yes",
    "TechSupport": "Yes",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Two year",             # 장기 계약
    "PaperlessBilling": "No",           # 종이 청구서
    "PaymentMethod": "Credit card (automatic)", # 자동이체 → 안정적
    "MonthlyCharges": 40.0,             # median 보다 낮음
    "TotalCharges": 800.0               # median 보다 낮음
}])


# 2. 이탈 확률 예측
y_prob = pipeline.predict_proba(new_customer)
print("예측 확률:", y_prob)   # 예: [[0.4486 0.5513]]
churn_pred = pipeline.predict(new_customer)[0]
print("예측 결과:", "Churn" if churn_pred==1 else "No Churn")

# 3. Risk 단계까지만 변환해서 파생변수 확인
X_risk = pipeline.named_steps["risk"].transform(new_customer)

# 4. Value 그룹 정의 (Median 기준 or 특정 Threshold)
median_monthly = X["MonthlyCharges"].median()
median_total   = X["TotalCharges"].median()

X_risk["ValueGroup"] = np.where(
    (X_risk["MonthlyCharges"] > median_monthly) | (X_risk["TotalCharges"] > median_total),
    "High Value", "Low Value"
)

# 5. Risk 그룹 정의 (RiskScore 활용)
X_risk["RiskGroup"] = np.where(X_risk["HighRisk"] == 1, "High Risk", "Low Risk")

# 6. 최종 세그먼트 조합
X_risk["Segment"] = X_risk["ValueGroup"] + " + " + X_risk["RiskGroup"]

print("최종 세그먼트:", X_risk["Segment"].iloc[0])

예측 확률: [[0.96831517 0.03168482]]
예측 결과: No Churn
최종 세그먼트: Low Value + Low Risk


In [483]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 인증 스코프
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"]

json_key_path = r"C:\Users\honor\OneDrive\Desktop\customer-crm-470315-59aca836b26d.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
client = gspread.authorize(creds)

# ✅ 새로운 스프레드시트 ID 사용
sheet = client.open_by_key("1LNBXDzzQzDznTBHTpc8bkFBZ1A36Ux5ll3j9kQNnoGQ").sheet1

# 데이터 읽기
records = sheet.get_all_records()
print(records[:3])


[{'gender': 'Male', 'SeniorCitizen': 0, 'Partner': 'No', 'Dependents': 'No', 'tenure': 13, 'PhoneService': 'No', 'MultipleLines': 'Yes', 'InternetService': 'DSL', 'OnlineSecurity': 'No', 'OnlineBackup': 'Yes', 'DeviceProtection': 'Yes', 'TechSupport': 'Yes', 'StreamingTV': 'No', 'StreamingMovies': 'Yes', 'Contract': 'Month-to-Month', 'PaperlessBilling': 'No', 'PaymentMethod': 'Mailed check', 'MonthlyCharges': 58.74, 'TotalCharges': 3609.56, 'Churn': 'No', 'Email': 'customer1@example.com'}, {'gender': 'Male', 'SeniorCitizen': 1, 'Partner': 'Yes', 'Dependents': 'No', 'tenure': 7, 'PhoneService': 'Yes', 'MultipleLines': 'No', 'InternetService': 'Fiber optic', 'OnlineSecurity': 'Yes', 'OnlineBackup': 'No', 'DeviceProtection': 'Yes', 'TechSupport': 'No', 'StreamingTV': 'Yes', 'StreamingMovies': 'No', 'Contract': 'Two year', 'PaperlessBilling': 'Yes', 'PaymentMethod': 'Bank transfer (automatic)', 'MonthlyCharges': 41.81, 'TotalCharges': 4791.87, 'Churn': 'Yes', 'Email': 'customer2@example.co

In [484]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(
    r"C:\Users\honor\OneDrive\Desktop\customer-crm-470315-59aca836b26d.json", scope
)
client = gspread.authorize(creds)

SHEET_ID = "1LNBXDzzQzDznTBHTpc8bkFBZ1A36Ux5ll3j9kQNnoGQ"
sheet = client.open_by_key(SHEET_ID).sheet1
df = pd.DataFrame(sheet.get_all_records())

y_prob = pipeline.predict_proba(df)[:, 1]
y_pred = pipeline.predict(df)

X_risk = pipeline.named_steps["risk"].transform(df)
df["Segment"] = np.where(X_risk["HighRisk"] == 1, "High Risk", "Low Risk")
df["ChurnProb"] = np.round(y_prob, 3)
df["ChurnPred"] = np.where(y_pred == 1, "Churn", "No Churn")


sheet.update([df.columns.values.tolist()] + df.values.tolist())




{'spreadsheetId': '1LNBXDzzQzDznTBHTpc8bkFBZ1A36Ux5ll3j9kQNnoGQ',
 'updatedRange': "'시트1'!A1:X21",
 'updatedRows': 21,
 'updatedColumns': 24,
 'updatedCells': 504}